# IbPy and Interactive Brokers Features Demonstration
Tested on Python 2

## Learning Outcomes
At the end of this simple workshop, you will be able to 
1. Extract Account and Portfolio Information
2. Placing Orders
3. Request Market Data
4. Obtain Historical Data
5. Access Market Depth Information
6. Download Real Time Bars
7. Extract Executions Information, including commission report

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
import time
from datetime import datetime
from IBWrapper import IBWrapper, contract
from ib.ext.EClientSocket import EClientSocket
from ib.ext.ScannerSubscription import ScannerSubscription

In the example to follow, **`callback`** is our `IBWrapper` instantiated. We **receive** information via `callback`.


In the example to follow, **`tws`** is our `EClientSocket` instantiated. We **request** information via `tws`

In [3]:
accountName = "DI246990"
callback = IBWrapper()             # Instantiate IBWrapper. callback 
tws = EClientSocket(callback)      # Instantiate EClientSocket and return data to callback
# host = "192.168.1.165"
host = ""
port = 4002
clientId = 5563

In [4]:
tws.eConnect(host, port, clientId) # Connect to TWS

Server Version: 76
TWS Time at connection:20180405 22:17:23 SGT
[-1, 2104, 'Market data farm connection is OK:hfarm']
[-1, 2104, 'Market data farm connection is OK:jfarm']
[-1, 2104, 'Market data farm connection is OK:eufarm']
[-1, 2104, 'Market data farm connection is OK:usfuture']
[-1, 2104, 'Market data farm connection is OK:cashfarm']
[-1, 2104, 'Market data farm connection is OK:usfarm.us']
[-1, 2104, 'Market data farm connection is OK:usfarm']
[-1, 2106, 'HMDS data farm connection is OK:ilhmds']
[-1, 2106, 'HMDS data farm connection is OK:euhmds']
[-1, 2106, 'HMDS data farm connection is OK:fundfarm']
[-1, 2106, 'HMDS data farm connection is OK:ushmds']


In [5]:
tws.setServerLogLevel(5)           # Set error output to verbose

In [6]:
create = contract()                # Instantiate contract class
callback.initiate_variables()

Note how the work flow goes. We send a request via the prefix **tws.** followed by the request for the specific type of data after the dot for the information we are interested in. 

For example, we would like to get an update on account time, which required us calling `reqAccountUpdates`. 

We request for info by calling **`tws.reqAccountUpdates`** and the data will be returned via our callback function. In this case **`callback.update_AccountTime`**

*****

# Account and Portfolio
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Account Updates
   * Account Value
   * Portfolio
   * Account Time
2. Account Summary
3. Positions

### Summary of Account and Portfolio 

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqAccountUpdates | updateAccountValue | self.update_AccountValue |
| | updatePortfolio | self.update_Portfolio |
| | updateAccountTime | self.update_AccountTime |
| reqAccountSummary | accountSummary | self.account_Summary |
| reqPositions | position | self.update_Position |

### Sending Account Updates Request
`reqAccountUpdates`

In [ ]:
tws.reqAccountUpdates(1, accountName)

#### Obtaining Account Value
`self.update_AccountValue`

In [ ]:
pd.DataFrame(callback.update_AccountValue, 
            columns = ['key', 'value', 'currency', 'accountName'])[:3]

#### Obtaining Portfolio Value
`self.update_Portfolio`

In [ ]:
pd.DataFrame(callback.update_Portfolio, 
             columns=['Contract ID','Currency',
                      'Expiry','Include Expired',
                      'Local Symbol','Multiplier',
                      'Primary Exchange','Right',
                      'Security Type','Strike',
                      'Symbol','Trading Class',
                      'Position','Market Price','Market Value',
                      'Average Cost', 'Unrealised PnL', 'Realised PnL', 
                      'Account Name'])[:3]

#### Obtaining Account Time
`self.update_AccountTime`

In [ ]:
callback.update_AccountTime

### Sending Account Summary Request
`reqAccountSummary`

This function call can only be made when connected to a Financial Advisor (FA) account. Another way to look at this is that if you have more than one account, use this function.

In [ ]:
tws.reqAccountSummary(2,"All","NetLiquidation")

#### Obtaining Account Summary
`self.account_Summary`

In [ ]:
pd.DataFrame(callback.account_Summary, 
             columns = ['Request_ID','Account','Tag','Value','Curency'])[:2]

### Sending Position Request
`reqPositions`

This function call request all positions for all accounts. This is more suitable for Financial Advisor. In the following example, I used pandas selection criteria to disply a specific account position.

In [ ]:
tws.reqPositions()

#### Obtaining Position
`self.update_Position`

In [ ]:
dat = pd.DataFrame(callback.update_Position, 
                   columns=['Account','Contract ID','Currency','Exchange','Expiry',
                            'Include Expired','Local Symbol','Multiplier','Right',
                            'Security Type','Strike','Symbol','Trading Class',
                            'Position','Average Cost'])
dat[dat["Account"] == accountName]

*****

# Orders
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Open Order
2. Next Valid ID
3. Order Status

### Summary of Orders

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqIds | nextValidId | self.next_ValidId |
| placeOrder | orderStatus | self.order_Status |
| cancelOrder | | |
| reqOpenOrders & reqAllOpenOrders | openOrder | self.open_Order |
| | orderStatus | self.order_Status |
| reqGlobalCancel | | |

#### Demo - Stock Purchase
* ** Request Next Valid Id**. `reqIds`
* ** Using Create**. `create`

In [7]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract("GOOG", "STK", "SMART", "USD")
order_info = create.create_order(accountName, "MKT", 250, "BUY")

#### Placing an Order
`placeOrder`

In [8]:
tws.placeOrder(order_id, contract_info, order_info)

#### Checking Order Status
`self.order_Status`

In [9]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,2,PreSubmitted,0,250,0.00,848701290,0,0.00,5563,locate
1,2,PreSubmitted,150,100,1034.45,848701290,0,1034.45,5563,locate
2,2,Submitted,150,100,1034.45,848701290,0,1034.45,5563,None
3,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
4,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
5,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None


## Limit Order

In [16]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract("GOOG", "STK", "SMART", "USD")
order_info = create.create_order(accountName, "LMT", 450, "BUY", 1000)

In [17]:
tws.placeOrder(order_id, contract_info, order_info)

In [18]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,2,PreSubmitted,0,250,0.00,848701290,0,0.00,5563,locate
1,2,PreSubmitted,150,100,1034.45,848701290,0,1034.45,5563,locate
2,2,Submitted,150,100,1034.45,848701290,0,1034.45,5563,None
3,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
4,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
5,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
6,5,Submitted,0,450,0.00,848701291,0,0.00,5563,None
7,5,Submitted,0,450,0.00,848701291,0,0.00,5563,None


#### Checking on Open Order
`self.open_Order`

In [19]:
callback.open_Order[:1]

[(2,
  <ib.ext.OrderState.OrderState at 0x113cadb00>)]

#### Cancelling Open Order
`cancelOrder`

In [20]:
tws.cancelOrder(order_id)

[5, 202, 'Order Canceled - reason:']


#### Demo
* ** Request Next Valid Id**. `reqIds`
* ** Using Create**. `create`
* ** Placing an Order to purchase Futures**. `placeOrder`

In [25]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract(symbol = "ES", secType = "FUT", 
                                       exchange = "GLOBEX", currency = "USD", 
                                       right = None, strike = None,
                                       expiry = "201806", multiplier=None,
                                       tradingClass=None)
order_info = create.create_order(accountName, "MKT", 1, "BUY")
tws.placeOrder(order_id, contract_info, order_info)

#### Checking Order Status

In [26]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,2,PreSubmitted,0,250,0.00,848701290,0,0.00,5563,locate
1,2,PreSubmitted,150,100,1034.45,848701290,0,1034.45,5563,locate
2,2,Submitted,150,100,1034.45,848701290,0,1034.45,5563,None
3,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
4,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
5,2,Filled,250,0,1034.45,848701290,0,1034.45,5563,None
6,5,Submitted,0,450,0.00,848701291,0,0.00,5563,None
7,5,Submitted,0,450,0.00,848701291,0,0.00,5563,None
8,5,Cancelled,0,450,0.00,848701291,0,0.00,5563,None
9,14,PreSubmitted,0,1,0.00,848701292,0,0.00,5563,locate


IB provided two more methods:
* `reqOpenOrders()` to request any open orders that were placed from this API client.
* `reqAllOpenOrders()` to request all open orders that were placed from all API clients linked to one TWS and also from the TWS.

Each open order will be fed back through `openOrder()` and `orderStatus()` methods.

Finally, use `reqGlobalCancel()` to cancel all open orders globally.

*****

# Market Data
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Market Data
   * Tick Price
   * Tick Size
2. Cancel Market Data
3. Calculate Implied Volatility
4. Calculate Option Price

### Summary of Market Data

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqMktData | tickPrice  | self.tick_Price |
|  | tickSize | self.tick_Size |
|  | tickOptionComputation  | self.tick_OptionComputation |
|  | tickGeneric | self.tick_Generic |
|  | tickString | self.tick_String |
|  | tickEFP  | self.tick_EFP |
|  | tickSnapshotEnd | self.tickSnapshotEnd_flag |
| cancelMktData | | |
| calculateImpliedVolatility | tickOptionComputation  | self.tick_OptionComputation |
| cancelcalculateImpliedVolatility | | |
| calculateOptionPrice  | tickOptionComputation  | self.tick_OptionComputation |
| cancelCalculateOptionPrice | | |
| reqMktDataType | marketDataType | self.market_DataType |

The method `reqMktDataType` allows you to toggle between receiving real-time or frozen market data.

#### Requesting Market Data
`reqMktData`

In [27]:
contract_info = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')
tickedId = 1002
tws.reqMktData(tickedId, contract_info, "", False)

#### Receiving Tick Price
`self.tick_Price`

In [28]:
tick_data = pd.DataFrame(callback.tick_Price, 
                         columns = ['tickerId', 'field', 'price', 'canAutoExecute'])
tick_type = {0 : "BID SIZE",
             1 : "BID PRICE",
             2 : "ASK PRICE",
             3 : "ASK SIZE",
             4 : "LAST PRICE",
             5 : "LAST SIZE",
             6 : "HIGH",
             7 : "LOW",
             8 : "VOLUME",
             9 : "CLOSE PRICE",
             10 : "BID OPTION COMPUTATION",
             11 : "ASK OPTION COMPUTATION",
             12 : "LAST OPTION COMPUTATION",
             13 : "MODEL OPTION COMPUTATION",
             14 : "OPEN_TICK",
             15 : "LOW 13 WEEK",
             16 : "HIGH 13 WEEK",
             17 : "LOW 26 WEEK",
             18 : "HIGH 26 WEEK",
             19 : "LOW 52 WEEK",
             20 : "HIGH 52 WEEK",
             21 : "AVG VOLUME",
             22 : "OPEN INTEREST",
             23 : "OPTION HISTORICAL VOL",
             24 : "OPTION IMPLIED VOL",
             27 : "OPTION CALL OPEN INTEREST",
             28 : "OPTION PUT OPEN INTEREST",
             29 : "OPTION CALL VOLUME"}
tick_data["Type"] = tick_data["field"].map(tick_type)
tick_data[-10:]

,tickerId,field,price,canAutoExecute,Type
0,1002,1,1.22441,1,BID PRICE
1,1002,2,1.22443,1,ASK PRICE
2,1002,6,1.22905,0,HIGH
3,1002,7,1.22355,0,LOW
4,1002,9,1.22780,0,CLOSE PRICE


#### Receiving Tick Size
`self.tick_Size`

In [29]:
tick_data = pd.DataFrame(callback.tick_Size, 
                         columns = ["tickerId", "field", "size"])
tick_data["Type"] = tick_data["field"].map(tick_type)
tick_data[-10:]

,tickerId,field,size,Type
184,1002,3,9400000,ASK SIZE
185,1002,0,3000000,BID SIZE
186,1002,3,10400000,ASK SIZE
187,1002,3,9400000,ASK SIZE
188,1002,3,8400000,ASK SIZE
189,1002,0,1000000,BID SIZE
190,1002,0,1000000,BID SIZE
191,1002,3,8900000,ASK SIZE
192,1002,0,4500000,BID SIZE
193,1002,0,4500000,BID SIZE


#### Calculate Implied Volatility
`calculateImpliedVolatility`

In [33]:
contract_info = create.create_contract(symbol='NFLX 180615C00300000',
                                       secType='OPT', exchange='SMART', 
                                       currency='USD',
                                       right='CALL', 
                                       strike='300', 
                                       expiry='20180615',
                                       multiplier=100, 
                                       tradingClass="NFLX")
tws.calculateImpliedVolatility(tickedId, 
                               contract_info, 
                               5.89, 
                               89.91)

In [34]:
pd.DataFrame(callback.tick_OptionComputation,
             columns=["tickerId", "field", "impliedVol", "delta",
                      "optPrice", "pvDividend", "gamma", "vega",
                      "theta", "undPrice"])

,tickerId,field,impliedVol,delta,optPrice,pvDividend,gamma,vega,theta,undPrice
0,1002,53,2.053619,9223372036854775807,5.89,9223372036854775807,9223372036854775807,9223372036854775807,9223372036854775807,89.91


#### Calculate Option Price
`tick_OptionComputation`

In [35]:
tws.calculateOptionPrice(tickedId, 
                         contract_info, 
                         0.84, 
                         89.91)

In [36]:
pd.DataFrame(callback.tick_OptionComputation,
             columns=["tickerId", "field", "impliedVol", "delta",
                      "optPrice", "pvDividend", "gamma", "vega",
                      "theta", "undPrice"])

,tickerId,field,impliedVol,delta,optPrice,pvDividend,gamma,vega,theta,undPrice
0,1002,53,2.053619,9.223372e+18,5.890000,9223372036854775807,9.223372e+18,9.223372e+18,9223372036854775807,89.91
1,1002,53,0.840000,9.649391e-04,0.008173,9223372036854775807,9.743840e-05,1.137006e-03,9223372036854775807,89.91


#### Cancelling Market Data Stream
`cancelMktData`

In [37]:
tws.cancelMktData(tickedId)

*****

# Historical Data
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Historical Data

### Summary of Historical Data

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqHistoricalData | historicalData  | self.historical_Data |

In [38]:
#contract_Details = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')
contract_Details = create.create_contract('AAPL', 'STK', 'SMART', 'USD')

In [39]:
data_endtime = datetime.now().strftime("%Y%m%d %H:%M:%S")

#### Requesting Historical Data
`reqHistoricalData`

In [40]:
tickerId = 9002
tws.reqHistoricalData(tickerId, 
                      contract_Details, 
                      data_endtime,
                      "1 M", 
                      "1 day", 
                      "BID", 
                      0, 
                      1)

In [42]:
data= pd.DataFrame(callback.historical_Data, 
                   columns = ["reqId", "date", "open",
                              "high", "low", "close", 
                              "volume", "count", "WAP", 
                              "hasGaps"])
data[-10:]

,reqId,date,open,high,low,close,volume,count,WAP,hasGaps
12,9002,20180323,167.74,169.92,164.87,165.15,-1,-1,-1.0,False
13,9002,20180326,167.60,173.10,166.42,172.91,-1,-1,-1.0,False
14,9002,20180327,174.01,175.13,167.01,168.65,-1,-1,-1.0,False
15,9002,20180328,163.32,170.00,163.32,166.85,-1,-1,-1.0,False
16,9002,20180329,167.10,171.74,166.71,168.17,-1,-1,-1.0,False
17,9002,20180402,167.60,168.94,164.47,166.55,-1,-1,-1.0,False
18,9002,20180403,167.50,168.73,164.88,167.75,-1,-1,-1.0,False
19,9002,20180404,165.10,171.99,163.40,171.93,-1,-1,-1.0,False
20,9002,20180405,165.06,173.55,165.06,172.36,-1,-1,-1.0,False
21,9002,finished-20180305 22:21:18-20180405 22:21:18,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.0,False


*****

# Market Depth
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Market Depth

### Summary of Market Depth

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqMktDepth           | updateMktDepth          | self.update_MktDepth |

In [43]:
contract_info = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')

In [44]:
tickerId = 7000
tws.reqMktDepth(tickerId, contract_info, 5)

[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']


In [45]:
operation_type = {0 : "Insert",
                  1 : "Update",
                  2 : "Delete",}
side_type = {0 : "Ask",
             1 : "Bid"}

In [46]:
data_mktdepth = pd.DataFrame(callback.update_MktDepth,
                             columns = ["tickerId", "position", 
                                        "operation", "side", 
                                        "price", "size"])
data_mktdepth["operation_type"] = data_mktdepth["operation"].map(operation_type)
data_mktdepth["side_type"] = data_mktdepth["side"].map(side_type)
data_mktdepth[-10:]

,tickerId,position,operation,side,price,size,operation_type,side_type
60,7000,2,1,1,1.22449,14000000,Update,Bid
61,7000,3,1,1,1.22448,4000000,Update,Bid
62,7000,2,1,1,1.22449,10500000,Update,Bid
63,7000,3,1,1,1.22448,5500000,Update,Bid
64,7000,1,1,0,1.22455,6000000,Update,Ask
65,7000,2,1,0,1.22456,4500000,Update,Ask
66,7000,2,1,1,1.22449,11500000,Update,Bid
67,7000,3,1,1,1.22448,2500000,Update,Bid
68,7000,2,1,1,1.22449,13000000,Update,Bid
69,7000,3,1,1,1.22448,1000000,Update,Bid


*****

# Real Time Bars
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Real Time Bars

Note:
* **barSize**. Only 5 sec bars are supported. 
* **whatToShow**:
   * TRADES
   * BID
   * ASK
   * MIDPOINT
* **useRTH**:
   * 0 = all data
   * 1 = only data within **R**egular **T**rading **H**ours

### Summary of Real Time Bars

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqRealTimeBars           | realtimeBar          | self.real_timeBar |

In [47]:
contract_Details = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')

In [48]:
tickerId = 10000
tws.reqRealTimeBars(tickerId, 
                    contract_Details, 
                    5, 
                    "MIDPOINT", 
                    0)

In [50]:
pd.DataFrame(callback.real_timeBar, 
             columns = ["reqId", "time", "open", "high", "low", "close", "volume", "wap", "count"])

,reqId,time,open,high,low,close,volume,wap,count
0,10000,1522938095,1.224530,1.224570,1.22452,1.224525,-1,-1.0,-1
1,10000,1522938100,1.224525,1.224525,1.22449,1.224510,-1,-1.0,-1


*****

# Executions
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Executions

### Summary of Executions

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqExecutions           | execDetails          | self.exec_Details_reqId |
| | | self.exec_Details_contract |
| | | self.exec_Details_execution |
| | execDetailsEnd |self.exec_DetailsEnd_flag |
| | commissionReport | self.commission_Report |

In [54]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract(symbol = "ES", secType = "FUT", 
                                       exchange = "GLOBEX", currency = "USD", 
                                       right = None, strike = None,
                                       expiry = "201806", multiplier=None,
                                       tradingClass=None)
order_info = create.create_order(accountName, "MKT", 1, "BUY")
tws.placeOrder(order_id, contract_info, order_info)

In [55]:
tws.reqExecutions(3050, create.exec_filter(clientId, accountName, contract_info))

In [56]:
callback.exec_Details_reqId

3050

In [57]:
callback.exec_Details_contract.__dict__

{'m_conId': 269745169,
 'm_currency': 'USD',
 'm_exchange': 'GLOBEX',
 'm_expiry': '20180615',
 'm_includeExpired': False,
 'm_localSymbol': 'ESM8',
 'm_multiplier': '50',
 'm_right': None,
 'm_secType': 'FUT',
 'm_strike': 0.0,
 'm_symbol': 'ES',
 'm_tradingClass': 'ES'}

In [58]:
callback.exec_Details_execution.__dict__

{'m_acctNumber': 'DI246990',
 'm_avgPrice': 2656.75,
 'm_clientId': 5563,
 'm_cumQty': 1,
 'm_evMultiplier': 0,
 'm_evRule': None,
 'm_exchange': 'GLOBEX',
 'm_execId': '0001f4e5.5ac59a6d.01.01',
 'm_liquidation': 0,
 'm_orderId': 21,
 'm_orderRef': None,
 'm_permId': 848701293,
 'm_price': 2656.75,
 'm_shares': 1,
 'm_side': 'BOT',
 'm_time': '20180405  22:22:10'}

In [59]:
pd.DataFrame(callback.commission_Report.__dict__, index=[0])

,m_commission,m_currency,m_execId,m_realizedPNL,m_yield,m_yieldRedemptionDate
0,2.05,USD,0001f4e5.5ac59a6d.01.01,1.797693e+308,1.797693e+308,0


In [60]:
tws.eDisconnect()

***